In [24]:
import sys
import itertools
from anytree import Node, RenderTree,Resolver,ChildResolverError,NodeMixin,PreOrderIter
from anytree.dotexport import RenderTreeGraph

class CustomNode(NodeMixin):  # Add state number feature
    def __init__(self, name, state_number,symbol,end_state, parent=None):
        super(CustomNode, self).__init__()
        self.name=name
        self.state_number=state_number
        self.parent=parent
        self.end_state=end_state
        self.symbol=symbol

state_number=0
top=CustomNode("root",state_number,'',False)
r = Resolver('name')




In [25]:
def add_inflectional_suffixes(suffix,state_number,root_node):
    suffixes=suffix.split(":")
    state_number+=1
    node=CustomNode('<eps>',state_number,'-',False,root_node)
    for ch in suffixes[0][1:]:
        state_number+=1
        node=CustomNode(ch,state_number,ch,False,node)
    state_number+=1
    CustomNode('<eps>',state_number,suffixes[1],True,node)
    if suffixes[1]=='+N':
        state_number+=1
        plural_node=CustomNode('^s',state_number,'+PL',True,node)
        state_number+=1
        CustomNode("^s",state_number,'+SGPS',True,node)
        state_number+=1
        CustomNode("'",state_number,'+PLPS',True,plural_node)
    elif suffixes[1]=='+V':
        state_number+=1
        CustomNode("^s",state_number,'+3D',True,node)
        state_number+=1
        CustomNode("ing",state_number,'+ING',True,node)
    return (node,state_number)
def append_suffixes(root_node,morphs,index,state_number):
    suffix=''
    while len(morphs)>index:
        index+=1
        if morphs[index]=='|':
            if suffix!='': #check if suffix to be handle before pipe
                
                node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
                suffix=''
        elif morphs[index]=='(':
            node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
            index,state_number=append_suffixes(node,morphs,index,state_number)
            suffix=''
        elif morphs[index]==')':
            if suffix!='': #check if suffix to be handle before paranthesis
                val=add_inflectional_suffixes(suffix,state_number,root_node)
                state_number=val[1]
            return (index,state_number)
        else:
            suffix+=morphs[index]
    return (index,state_number)

def insert_irregular_verb(stem,state_number,root):
    top=root
    verbs=stems=stem.split('/')
    for i in range(len(verbs[0])):
        state_number+=1
        root=CustomNode(verbs[0][i],state_number,verbs[0][i],False,root)
    end_ptr=root 
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+V',True,end_ptr)
    
    # add past tense
    end_ptr=top  
    for i in range(len(verbs[0])):
        state_number+=1
        end_ptr=CustomNode('<eps>',state_number,verbs[0][i],False,end_ptr)
    for i in range(len(verbs[1])):
        state_number+=1
        end_ptr=CustomNode(verbs[1][i],state_number,'<eps>',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+V',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+PT',True,end_ptr)    
    
    #add past perfect
    end_ptr=top
    for i in range(len(verbs[0])):
        state_number+=1
        end_ptr=CustomNode('<eps>',state_number,verbs[0][i],False,end_ptr)
    for i in range(len(verbs[2])):
        state_number+=1
        end_ptr=CustomNode(verbs[2][i],state_number,'<eps>',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+V',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+PP',True,end_ptr)    
            
    return (root,state_number)

In [26]:
#words = ['compute (er:+N(ize:+V(ation:+N|able:+ADJ)|less:+ADJ(ness:+N))|ation:+N(able:+ADJ))','work (er:+N(ism:+N))']
#words = ["be/were&was/been (ing:+N)","begin/began/begun (ing:+N|er:+N)"]
with open('Lexicon/Verbs.txt') as f:
    words = f.readlines()
    words.sort()
    for word in words:
        line=word.rstrip()
        arr=line.split(' ')
        stem=arr[0]
        if len(arr)==2:
            morphs=arr[1]
        root=top

        if '/' in stem: #means irregular verb
            root,state_number=insert_irregular_verb(stem,state_number,top)
        else:
            for i in range(len(stem)):
                try:
                    n=r.get(root,stem[i])
                except ChildResolverError:
                    state_number+=1
                    n=CustomNode(stem[i],state_number,stem[i],False,root)
                root=n
            state_number+=1
            CustomNode("<eps>",state_number,'+V',True,root)
            state_number+=1
            CustomNode("^s",state_number,'+3D',True,root)
            state_number+=1
            CustomNode("ing",state_number,'+ING',True,root)
        if len(arr)==2:
            return_val=append_suffixes(root,morphs,0,state_number)
            state_number=return_val[1]
RenderTreeGraph(top).to_picture("udo.png")

In [27]:
p=PreOrderIter(top)

p=itertools.islice(p,1,None) #pass root


for n in p:
    print(n.name + " " +str(n.state_number) + "parent: " + n.parent.name)

a 1parent: root
c 2parent: a
c 3parent: c
e 4parent: c
p 5parent: e
t 6parent: p
<eps> 7parent: t
^s 8parent: t
ing 9parent: t
<eps> 10parent: t
e 11parent: <eps>
e 12parent: e
<eps> 13parent: e
^s 14parent: e
' 16parent: ^s
^s 15parent: e
<eps> 17parent: t
i 18parent: <eps>
n 19parent: i
g 20parent: n
<eps> 21parent: g
<eps> 22parent: g
n 23parent: <eps>
e 24parent: n
s 25parent: e
s 26parent: s
<eps> 27parent: s
^s 28parent: s
' 30parent: ^s
^s 29parent: s
<eps> 31parent: g
l 32parent: <eps>
y 33parent: l
<eps> 34parent: y
<eps> 35parent: t
a 36parent: <eps>
t 37parent: a
i 38parent: t
o 39parent: i
n 40parent: o
<eps> 41parent: n
^s 42parent: n
' 44parent: ^s
^s 43parent: n
<eps> 45parent: t
o 46parent: <eps>
r 47parent: o
<eps> 48parent: r
^s 49parent: r
' 51parent: ^s
^s 50parent: r
<eps> 52parent: t
e 53parent: <eps>
n 54parent: e
c 55parent: n
e 56parent: c
<eps> 57parent: e
^s 58parent: e
' 60parent: ^s
^s 59parent: e
<eps> 61parent: t
e 62parent: <eps>
n 63parent: e
c 64parent

In [28]:

#for pre, _, node in RenderTree(top):
#    treestr = u"%s%s" % (pre, node.name)
#    print(treestr.ljust(8), node.state_number)
def print_as_tabs(prev_state,next_state,input_label,output_label):
    print(str(prev_state) + "\t" + str(next_state) + "\t"+ str(input_label) + "\t" + str(output_label))


# In[222]:


p=PreOrderIter(top)
p=itertools.islice(p,1,None) #pass root

print_as_tabs(0,1,"<s>","<s>")
print_as_tabs(1,0,"<eps>","<eps>")
print_as_tabs(0,2,"</s>","</s>")
print_as_tabs(2,0,"<eps>","<eps>")
print_as_tabs(0,3,"<unk>","<unk>")
print_as_tabs(3,0,"<eps>","<eps>")
for n in p:
    parent_state_number=n.parent.state_number
    if n.parent.state_number!=0:
        parent_state_number+=3
    print_as_tabs(parent_state_number,n.state_number+3,n.symbol,n.name)
    if n.end_state:
        print_as_tabs(n.state_number+3,0,"<eps>","#")  
    #if n.is_leaf:
     #   print_as_tabs(n.state_number+3,0,"<eps>","#")
print("0")


0	1	<s>	<s>
1	0	<eps>	<eps>
0	2	</s>	</s>
2	0	<eps>	<eps>
0	3	<unk>	<unk>
3	0	<eps>	<eps>
0	4	a	a
4	5	c	c
5	6	c	c
6	7	e	e
7	8	p	p
8	9	t	t
9	10	+V	<eps>
10	0	<eps>	#
9	11	+3D	^s
11	0	<eps>	#
9	12	+ING	ing
12	0	<eps>	#
9	13	-	<eps>
13	14	e	e
14	15	e	e
15	16	+N	<eps>
16	0	<eps>	#
15	17	+PL	^s
17	0	<eps>	#
17	19	+PLPS	'
19	0	<eps>	#
15	18	+SGPS	^s
18	0	<eps>	#
9	20	-	<eps>
20	21	i	i
21	22	n	n
22	23	g	g
23	24	+ADJ	<eps>
24	0	<eps>	#
23	25	-	<eps>
25	26	n	n
26	27	e	e
27	28	s	s
28	29	s	s
29	30	+N	<eps>
30	0	<eps>	#
29	31	+PL	^s
31	0	<eps>	#
31	33	+PLPS	'
33	0	<eps>	#
29	32	+SGPS	^s
32	0	<eps>	#
23	34	-	<eps>
34	35	l	l
35	36	y	y
36	37	+ADV	<eps>
37	0	<eps>	#
9	38	-	<eps>
38	39	a	a
39	40	t	t
40	41	i	i
41	42	o	o
42	43	n	n
43	44	+N	<eps>
44	0	<eps>	#
43	45	+PL	^s
45	0	<eps>	#
45	47	+PLPS	'
47	0	<eps>	#
43	46	+SGPS	^s
46	0	<eps>	#
9	48	-	<eps>
48	49	o	o
49	50	r	r
50	51	+N	<eps>
51	0	<eps>	#
50	52	+PL	^s
52	0	<eps>	#
52	54	+PLPS	'
54	0	<eps>	#
50	53	+SGPS	^s
53	0	<eps>	#
9	55	-	<eps>
55	56	e	e
56	5

In [30]:
def append_suffixes(node_stack,morphs,index,paranthesis_stack,state_number):
    suffix=''
    while len(paranthesis_stack)!=0:
        index+=1
        
        if morphs[index]=='|':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
                suffix=''
        elif morphs[index]=='(':
            suffixes=suffix.split(':')
            state_number+=1
            
            node=CustomNode(suffixes[0],state_number,suffixes[1],node_stack[-1])
            node_stack.append(node)
            paranthesis_stack.append(index)
            param_node_stack=[node_stack[-1]]
            param_paranthesis_stack=[index]
            print(node_stack[-1].name)
            print(param_paranthesis_stack)
        
            var=append_suffixes(param_node_stack,morphs,index,param_paranthesis_stack,state_number)
            index=var[0]
            state_number=var[1]
            node_stack.pop()
            paranthesis_stack.pop()
            suffix=''
            print(node_stack[-1].name)
        elif morphs[index]==')':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
            paranthesis_stack.pop()
            node_stack.pop()
            return (index,state_number,node_stack,paranthesis_stack)
        else:
            suffix+=morphs[index]
    return (index,state_number,node_stack,paranthesis_stack)